# CA3 model

This notebook is for building out the CA3 model as previous implementation doesn't work.

Formula is:
$$
min_{\vec{w}_{x_1}, \vec{w}_{x_2}, \vec{w}_a, \vec{w}_b} \left( -\vec{w}_{x_1}^T S_{xa} \vec{w}_a - \vec{w}_{x_2}^T S_{xb} \vec{w}_b + \sum_{i \in \{x_1, x_2, a, b\}} \frac{1}{2} \lambda_i \left( \vec{w}_i^T S_{ii} \vec{w}_i - 1 \right) + \theta_{rr}(\vec{w}_{x_1}, \vec{w}_{x_2}) \right)
$$


In [1]:
from gemmr.generative_model import GEMMR
import numpy as np
from scipy.optimize import minimize
import itertools
import scipy.stats

## Generate some data

import numpy as np
from sklearn.cross_decomposition import CCA

np.random.seed(0)

n_samples = 200
n_features_x = 10
n_features_y = 8

# Latent variables (shared source of variance)
latent = np.random.randn(n_samples, 3)

# Mix latent variables with some noise
behavioural_data_study1 = latent @ np.random.randn(3, n_features_x) + 0.1 * np.random.randn(n_samples, n_features_x)
imging_data_study1 = latent @ np.random.randn(3, n_features_y) + 0.1 * np.random.randn(n_samples, n_features_y)


In [160]:
model_definition = GEMMR('cca', wx=2, wy=50, r_between=0.3)
behavioural_data_study1, imging_data_study1 = model_definition.generate_data(n=200)
behavioural_data_study2, imging_data_study2 = model_definition.generate_data(n=190)
study1 = (imging_data_study1, behavioural_data_study1) 
study2 = (imging_data_study2, behavioural_data_study2)

In [3]:
from gemmr.estimators import SVDCCA
cca = SVDCCA().fit_transform(imging_data_study1, behavioural_data_study1)
len(cca[1])

200

In [4]:
r2= []
for val in range(imging_data_study1.shape[1]):
    corr = scipy.stats.pearsonr(behavioural_data_study1[:, 1], imging_data_study1[:, val])[0]
    r2.append(corr)
np.mean(r2)

np.float64(0.008702029962619288)

## Step 1: Covariance matrix

In [5]:
def mean_center(data: np.ndarray) -> np.ndarray:
    """
    Function to demean data.

    Parmeteres
    ----------
    data: np.ndarray
        data to demean

    Returns
    -------
    np.ndarray: array
        demeaned data
    """
    return data - data.mean(axis=0)

In [6]:
def cross_cov(matrix_1: np.ndarray, matrix_2: np.ndarray) -> np.ndarray:
    """
    Function to calculate 
    covariance matrix

    Parameters
    ----------
    matrix_1: np.ndarray
        A matrix tht should 
        correspond to subject by 
        features
    matrix_2: np.ndarray
        A matrix that should 
        correspond to features by
        feautres 

    Returns
    -------
    np.ndarray: array
        array of cross covariance matrix
    """
    return (matrix_1.T @ matrix_2) / matrix_1.shape[0] 

In [7]:
def data_able_to_process(study_pair: tuple, behav_data: np.ndarray, img_data: np.ndarray) -> bool:
    """
    Function to check that data
    is in correct format to be processed

    Parameters
    ----------
     study_pair: tuple, 
         tuple of behavioural data 
         and imging data
     behav_data: np.ndarray
         array of behav_data 
     img_data: np.ndarray
         array of img_data
    
    Returns
    -------
    bool: boolean
        bool of if failed or not
    """
    if not isinstance(study_pair, (tuple, list)) or len(study_pair) != 2:
        print("Given argument isn't a pair of datasets")
        return False
    if not isinstance(behav_data, np.ndarray) or not isinstance(img_data, np.ndarray):
        print("Data provided isn't a numpy array")
        return False
    if behav_data.shape[0] == 0 or img_data.shape[0] == 0 or behav_data.shape[0] != img_data.shape[0]:
        print(f"Mismatch between ({behav_data.shape[0]} and {img_data.shape[0]})")

    return True

In [8]:
def calculate_covariance_matricies(*study_pairs) -> dict:
    """
    Calculates covariance matrices and auto covariance
    matricies

    Parameters
    ----------
    study_pairs: tuple
        a tuple or list containing two numpy arrays:
        (behavioural_data, imaging_data).
        Assumes data is (subjects x features).

    Returns
    -------
    covariance_results: dict
        dictionary of covariance and auto-covariance matrices

    """
    covariance_results = {}
    for idx, study_pair in enumerate(study_pairs):
        behav_data, img_data = study_pair
        if not data_able_to_process(study_pair, behav_data, img_data):
            continue
        behav_data = mean_center(behav_data)
        img_data = mean_center(img_data)
        study_num = idx + 1
        try:
            covariance_results[f"s_behav{study_num}_behav{study_num}"] = cross_cov(behav_data, behav_data)
            covariance_results[f"s_img{study_num}_img{study_num}"] = cross_cov(img_data, img_data)
            covariance_results[f"s_img{study_num}_behav{study_num}"] = cross_cov(img_data, behav_data)

        except Exception as e:
            print(f"Error calculating covariances for Study {study_num}: {e}")
            return None
    return covariance_results

In [9]:
covariance_mat = calculate_covariance_matricies(study1, study2)

In [10]:
s_x1a = cross_cov(behavioural_data_study1, imging_data_study1)
s_x2b = cross_cov(behavioural_data_study2, imging_data_study2)
s_x1x1 = cross_cov(behavioural_data_study1, behavioural_data_study1)
s_x2x2 = cross_cov(behavioural_data_study2, behavioural_data_study2)
s_aa = cross_cov(imging_data_study1, imging_data_study1)
s_bb = cross_cov(imging_data_study2, imging_data_study2)

## Step 2. Intialization of weights 

In [11]:
def weight_intialization(*weights) -> np.ndarray:
    """
    Define a set of random starting 
    weights
    
    Parameters
    ----------
    weights: tuple(int)
        tuple of set amount
        of int values
    
    Returns
    -------
    np.ndarrray
        array of numpy values
    """
    return np.random.randn(sum(weights))

In [12]:
dim = {
  '1': behavioural_data_study1.shape[1], 
   '2': behavioural_data_study2.shape[1],
    '3': imging_data_study1.shape[1],
    '4': imging_data_study2.shape[1]
 }

In [13]:
weights_0 = weight_intialization(
    behavioural_data_study1.shape[1], 
    behavioural_data_study2.shape[1],
    imging_data_study1.shape[1],
    imging_data_study2.shape[1]
    )

## Step 3. Objective function

In [14]:
dx1_shape = s_x1x1.shape[0]
dx2_shape = s_x2x2.shape[0]
da_shape = s_aa.shape[0]
db_shape = s_bb.shape[0]
dx_shape = dx1_shape + dx2_shape
dac_shape =  dx_shape + da_shape

def get_dimensions(s_x1x1, s_x2x2, s_aa):
    dx1_shape = s_x1x1.shape[0]
    dx2_shape = s_x2x2.shape[0]
    da_shape = s_aa.shape[0]
    dx_shape =  dx1_shape + dx2_shape
    return {
        'dx1_shape': dx1_shape,
        'dx_shape' : dx_shape,
        'dac_shape':  dx_shape + da_shape
    }

def get_weights(weight_array, dx1_shape, dx_shape, dac_shape):
    return {
        "wx1": weight_array[:dx1_shape],
        "wx2": weight_array[dx1_shape:dx_shape],
        "wa": weight_array[dx_shape:dac_shape],
        "wb": weight_array[dac_shape:]
    }


In [15]:
def cross_cov_term(weight_beh, cov_mat, weight_img):
    return -weight_beh.T @ cov_mat @ weight_img

In [16]:
def regularization_term(weight, cov_mat):
    return 0.5 * 1.0 * (weight.T @ cov_mat @ weight - 1)

In [17]:
def dissimilarity_penality(theta_r, img_weight1, img_weight2):
    return 0.5 * theta_r * np.sum((img_weight1 - img_weight2) ** 2)


In [18]:
def objective_function(weights, s_x1a, s_x2b, s_x1x1, s_x2x2, s_aa, s_bb, theta_r):
    dimensions = get_dimensions(s_x1x1, s_x2x2, s_aa)
    weights = get_weights(
            weights, 
            dimensions['dx1_shape'], 
            dimensions['dx_shape'], 
            dimensions['dac_shape'])
    term1 = cross_cov_term(weights['wx1'],s_x1a, weights['wa'])
    term2 = cross_cov_term(weights['wx2'],s_x2b, weights['wb'])
    reg_x1 = regularization_term(weights['wx1'], s_x1x1)
    reg_x2 = regularization_term(weights['wx2'], s_x2x2)
    reg_a = regularization_term(weights['wa'], s_aa)
    reg_b = regularization_term(weights['wb'], s_bb)
    theta_r = dissimilarity_penality(theta_r, weights['wa'], weights['wb'])
    return term1 + term2 + reg_x1 + reg_x2 + reg_a + reg_b + theta_r


## Step 4: Minimise

In [19]:
best_loss = float('inf')
optimal_theta_r = None
optimium_model = None

for theta_r in np.logspace(-3, 2, 10):
    weights_0 = weight_intialization(
        behavioural_data_study1.shape[1], 
        behavioural_data_study2.shape[1],
        imging_data_study1.shape[1],
        imging_data_study2.shape[1])  # re-init each time
    res = minimize(
        objective_function,
        weights_0,
        args=(s_x1a, s_x2b, s_x1x1, s_x2x2, s_aa, s_bb, theta_r),
        method='L-BFGS-B'
    )
    if res.status !=0:
        print(res.status)
        continue


    if res.fun < best_loss:
        best_loss = res.fun
        best_theta_r = theta_r
        optimium_model = res

print(f"Best θ_r: {best_theta_r}")
print(f"Best loss: {best_loss}")

1
1
1
Best θ_r: 0.1668100537200059
Best loss: -1.9999999932695567


In [20]:
optimium_model

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: -1.9999999932695567
        x: [-4.030e-05  7.906e-06 ...  1.845e-06 -2.273e-04]
      nit: 50
      jac: [-4.192e-05 -4.485e-06 ... -5.773e-07 -3.841e-06]
     nfev: 5775
     njev: 55
 hess_inv: <104x104 LbfgsInvHessProduct with dtype=float64>

In [21]:
dimensions = get_dimensions(s_x1x1, s_x2x2, s_aa)
weights = get_weights(optimium_model.x,
            dimensions['dx1_shape'], 
            dimensions['dx_shape'], 
            dimensions['dac_shape'])

In [22]:
# Get projections (scores)
scores_x1 = behavioural_data_study1 @ weights['wx1']
scores_x2 = behavioural_data_study2 @ weights['wx2']
scores_a  = imging_data_study1 @ weights['wa']
scores_b  = imging_data_study2 @ weights['wb']


In [23]:
display(np.corrcoef(scores_x1, scores_a)[0, 1])
display(np.corrcoef(scores_x2, scores_b)[0, 1])
display(np.linalg.norm(weights['wa'] - weights['wb']))

np.float64(0.032719708577966966)

np.float64(0.012496034191626546)

np.float64(5.5478433431911584e-05)

## Putting it all together

In [165]:
class CA3:
    def __init__(self, theta: float=None):
        self.theta_ = np.logspace(-3, 2, 10) if theta is None else theta
        self.intial_weights_ = None
        self.dims_ = []
        self.best_loss = float('inf')
        self.optimal_theta_r = None
        self.weights_ = None
 

    def fit(self, *data_sets):
        self.covariances_ = calculate_covariance_matricies(*data_sets)
        self.dims_ = self._get_dimensions(*data_sets)
        self.intial_weights_ = weight_intialization(*list(itertools.chain(*self.dims_)))
        self._optimise()

    def transform(self, *data_sets):
        assert self.weights_ is not None, "Model must be fitted before transfomed can be called."
        assert len(data_sets) == len(self.dims_), "Model fitted with different number of datasets."

        scores = []
        correlations = []
    
        for (X_img, X_behav), (wx, wb) in zip(data_sets, self.weights_):
            # Project to canonical components
            print(wx)
            u = X_img @ wx
            v = X_behav @ wb
            scores.append((u, v))
    
            # Correlation coefficients per component
            corr = np.array([np.corrcoef(u[:, i], v[:, i])[0, 1] for i in range(u.shape[1])])
            correlations.append(corr)
    
        return correlations, scores
 
            
    def _optimise(self):
        if isinstance(self.theta_, (list, np.ndarray)):
            for theta in self.theta_:
                model = self._optimising_model(theta)
                if model.status !=0:
                    continue
                if model.fun < self.best_loss:
                    self.best_loss = model.fun
                    self.optimal_theta_r = theta
                    self.weights_ = self._split_weights(model.x)
        else:
            model = self._optimising_model(theta)
            self.best_loss = model.fun
            self.optimal_theta_r = theta
            self.weights_ = self._split_weights(model.x)


    def _optimising_model(self, theta):
        return minimize(
            self._objective_function,
            self.intial_weights_,
            args=(self.covariances_, theta),
            method='L-BFGS-B'
        )
    
    def _get_dimensions(self, *data_sets):
        return [(behav.shape[1], img.shape[1]) for behav, img in data_sets]

    
    def _split_weights(self, w):
        """
        Splits the flat weight vector w into individual vectors
        for each behavioural and imaging dataset.
        """
        offset = 0
        weights = []
        for behav_dim, img_dim in self.dims_:
            wx = w[offset:offset + img_dim]
            offset += img_dim
            wb = w[offset:offset + behav_dim]
            offset += behav_dim
            weights.append((wx, wb))
        return weights
    
    def _objective_function(self, weights, covariances, theta):
        total_loss = 0
        weights_ = self._split_weights(weights)
        for idx, (wx, wb) in enumerate(weights_):

            s_xb = covariances[f"s_img{idx+1}_behav{idx+1}"]
            s_xx = covariances[f"s_img{idx+1}_img{idx+1}"]
            s_bb = covariances[f"s_behav{idx+1}_behav{idx+1}"]
    
            total_loss += cross_cov_term(wx, s_xb, wb) 
            total_loss += regularization_term(wx, s_xx)
            total_loss += regularization_term(wb, s_bb)
    
        # Similarity penalty across imaging weights
        if theta > 0 and len(weights_) > 1:
            for img_data in range(len(weights_)):
                for next_img_data in range(img_data + 1, len(weights_)):
                    total_loss += dissimilarity_penality(theta, weights_[img_data][0], weights_[next_img_data][0])
    
        return total_loss

In [166]:
ca3 = CA3()
ca3.fit(study1, study2)
#ca3.transform(study1, study2)


In [167]:
ca3.transform(study1, study2)

[ 1.08737774e-05 -1.13659419e-05]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 50)

In [172]:
for (X_img, X_behav), (wx, wb) in zip((study1, study2), ca3.weights_):
    print(X_img.shape)
    print(wb.shape)


(200, 50)
(50,)
(190, 50)
(50,)
